In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

import torch.nn as nn

from src.consts import *
from src.corpus import Corpus
from src.model import RNNModel
from src.training import train, evaluate
from src.main import setup_torch, get_corpus
from src.custom_data_parallel import CustomDataParallel

from notebooks.utils import summary, check_cuda_mem

In [3]:
setup_torch()
device = torch.device("cuda:0" if USE_CUDA else "cpu")
corpus = get_corpus()

In [4]:
assert len(corpus.dictionary) == 602755
assert corpus.valid.size()[0] == 11606861

In [5]:
ntokens = len(corpus.dictionary)
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
                 TIED)
criterion = nn.CrossEntropyLoss()
model = CustomDataParallel(model)

In [9]:
train(model, corpus, criterion, device)

INFO 2019-05-22 17:50:30,604: | epoch   1 |   200/23759 batches | lr 20.00 | ms/batch 1497.00 | loss  8.12 | ppl  3358.49
INFO 2019-05-22 17:55:28,476: | epoch   1 |   400/23759 batches | lr 20.00 | ms/batch 1489.36 | loss  8.07 | ppl  3201.52
INFO 2019-05-22 17:56:42,932: -----------------------------------------------------------------------------------------
INFO 2019-05-22 17:56:42,933: Exiting from training early
/home/gabrielamelo/anaconda3/envs/wsc_port/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'src.model.RNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


TypeError: evaluate() missing 3 required positional arguments: 'corpus', 'criterion', and 'device'

In [8]:
with open(MODEL_FILE_NAME, 'wb') as f:
    torch.save(model, f)

/home/gabrielamelo/anaconda3/envs/wsc_port/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyDataParallel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
